# Hearthstone Project
***
***

# Goals
- Thoroughly prepare so that it is ready for exploration and modeling
    - I won't be creating any models for this project but I'd still like to prepare the data nonetheless
- Explore the data to gather insights about the characteristics of the game's different class types

# Setup
***

In [1]:
# establishing environment
import pandas as pd
import numpy as np

# Acquire
Acquiring data from local csv files
***

In [2]:
# reading in card data and saving as DF
cards = pd.read_csv('hearthstone_standard_cards.csv')

# reading in card classes and saving as DF
classes = pd.read_csv('classes.csv')

# reading in minion types data and saving as DF
mtypes = pd.read_csv('minionTypes.csv')

# reading in rarities data and saving as DF
rarities = pd.read_csv('rarities.csv')

# reading in set groups data and saving as DF
setgroups = pd.read_csv('setGroups.csv')

# reading in card sets data and saving as DF
sets = pd.read_csv('sets.csv')

# reading in types data and saving as DF
ctypes = pd.read_csv('types.csv')

# reading in keywords data and saving as DF
keywords = pd.read_csv('keywords.csv')

# Prepare
Preparing data for exploration
***

## Merging Data frames

### Merging 'classes' DF

In [3]:
# merging 'classes' df
df = pd.merge(cards, classes[['id', 'slug']], 
              left_on = 'classId', right_on = 'id', how="left", 
              suffixes = (None, '_hero_class'))

### Merging 'mtypes' DF

In [4]:
# merging 'mtypes' df
df = pd.merge(df, mtypes[['id', 'slug']], 
              left_on = 'minionTypeId', right_on = 'id', how="left", 
              suffixes = (None, '_minion_type_id'))

### Merging 'rarities' DF

In [5]:
# merging 'rarities' df
df = pd.merge(df, rarities[['id', 'slug']], 
              left_on = 'rarityId', right_on = 'id', how="left", 
              suffixes = (None, '_rarity'))

### Merging 'setGroups' DF

In [6]:
# merging 'setgroups' df
df = pd.merge(df, sets[['id', 'slug', 'name']], 
              left_on = 'cardSetId', right_on = 'id', how="left", 
              suffixes = (None, '_set'))

### Merging 'ctypes' DF

In [7]:
# merging 'ctypes' df
df = pd.merge(df, ctypes[['id', 'slug']], 
              left_on = 'cardTypeId', right_on = 'id', how="left", 
              suffixes = (None, '_card_type'))

### Adding missing keyword to 'keywords' DF

In [8]:
keywords.loc[len(keywords.index)] = ['64', 'start-of-game', 'Start of Game', 
                                     'Does something at the start of the Game.', 
                                     'Does something at the start of the Game.']

### Merging 'keywords' DF

In [9]:
# removing brackets and commas from keyword id column
df.keywordIds = df.keywordIds.str.replace('\]|,|\[' , '')

# splitting keyword ids into separate columns for each card
kwdf = df["keywordIds"].str.split(" ", expand = True) 

# renaming columns
kwdf.columns = ['keywordId1', 'keywordId2', 'keywordId3', 'keywordId4', 'keywordId5']

# concatenating split keyword id columns with main df
df = pd.concat([df, kwdf], axis=1)

# converting keywords id column to str type to enable merge
keywords.id = keywords.id.astype(str)

# creating loop to add a column for the text name of each keyword ability of each card
# via merging with keywords DF
for x in kwdf.columns:
    df = pd.merge(df, keywords[['id', 'slug']], 
              left_on = x, right_on = 'id', how = "left",
              suffixes = (None, x + '_name'))

## Checking for duplicate rows

In [10]:
# checking number of rows in current DF
df.shape

(1289, 50)

In [11]:
# Checking number of rows if duplicates were dropped
df.drop_duplicates().shape

(1289, 50)

- No duplicates found

### Checking for proper data types, categorical columns (based on domain knowledge), and null counts

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1289 entries, 0 to 1288
Data columns (total 50 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1289 non-null   int64  
 1   collectible          1289 non-null   int64  
 2   slug                 1289 non-null   object 
 3   classId              1289 non-null   int64  
 4   multiClassIds        1289 non-null   object 
 5   cardTypeId           1289 non-null   int64  
 6   cardSetId            1289 non-null   int64  
 7   rarityId             1289 non-null   int64  
 8   artistName           1288 non-null   object 
 9   manaCost             1289 non-null   int64  
 10  name                 1289 non-null   object 
 11  text                 1271 non-null   object 
 12  image                1289 non-null   object 
 13  imageGold            805 non-null    object 
 14  flavorText           1289 non-null   object 
 15  cropImage            1289 non-null   o

- The following columns will be dropped as they won't be needed for the expected operations of this project
    - id, slug
        - unique identifiers for cards, not needed since the 'name' column provides this while also being easier to reference
    - artistName, image, imageGold, cropImage
        - I won't be exploring images or artist names in this iteration of the project
    - all columns reflecting key words with the exception of the boolean columns and the 'slug_keyword#_name' columns
        - The exempted columns are sufficient for the project's expected operations
     
     
- Based on my domain knowledge of the game, I'm inferring that several of the columns are categorical
    - I need to create boolean columns for categorical columns (rarity, card set, etc.)


- Many null values that need to be addressed
    - text
    - duels
    - minion type id
    - health
    - attack
    - child ids
    - durability
    - armor

### Dropping columns that aren't needed for the expected operations of this project

In [13]:
# creating list of columns to drop
columns_to_drop = ['id', 'slug', 'artistName', 'image', 'imageGold', 'flavorText', 'cropImage']

# dropping columns
df.drop(columns = columns_to_drop, inplace = True)

# Creating boolean columns for categorical variables

### Creating boolean columns for keywords

In [14]:
# loop iterates through each keyword and creates a boolean column for it
for kw in keywords.slug:
    df['has_' + kw] = np.where(
    (df.slugkeywordId1_name == kw) |
    (df.slugkeywordId2_name == kw) |
    (df.slugkeywordId3_name == kw) |
    (df.slugkeywordId4_name == kw) |
    (df.slugkeywordId5_name == kw), 1, 0)

### Dropping keyword related columns that are no longer needed 

In [15]:
# creating empty list
key_word_col_drop = []

# iterating through columns in df and creating list of columns to drop
for col in df.columns:
    if 'keywordId' in col:
        key_word_col_drop.append(col)
        
# dropping columns
df.drop(columns = key_word_col_drop, inplace = True)

### Creating boolean columns for class types

In [16]:
# removing brackets and commas from multiclassids column
df.multiClassIds = df.multiClassIds.str.replace('\]|,|\[' , '')

# creating column that holds secondary class separate from primary class
df['classId2'] = df["multiClassIds"].str.split(" ", expand = True)[1]

# converting column to str type to enable merge with newly created column 'classId2'
classes.id = classes.id.astype(str)

# creating df containing columns for merge in order rename before merge without altering original classes DF
classes2 = classes[['id', 'slug']]

# renaming columns
classes2.columns = ['classId2', 'slug_hero_class2']

# merging 'classes' on secondary hero class id to get secondary class names
df = pd.merge(df, classes2[['classId2', 'slug_hero_class2']], 
              on = 'classId2', how="left")

# creating boolean columns for each hero class
for c in classes.slug:
    df['is_' + c] = np.where(
    (df.slug_hero_class == c) | (df.slug_hero_class2 == c), 1, 0)

In [47]:
df[df.name == 'Cutting Class']

,collectible,classId,multiClassIds,cardTypeId,cardSetId,rarityId,manaCost,name,text,duels,...,is_druid,is_hunter,is_mage,is_paladin,is_priest,is_rogue,is_shaman,is_warlock,is_warrior,is_neutral
1184,1,12,7 10,5,1443,1,5,Cutting Class,Draw 2 cards. Costs (1) less per Attack of you...,"{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,1,1


In [48]:
df

,collectible,classId,multiClassIds,cardTypeId,cardSetId,rarityId,manaCost,name,text,duels,...,is_druid,is_hunter,is_mage,is_paladin,is_priest,is_rogue,is_shaman,is_warlock,is_warrior,is_neutral
0,1,14,,5,1463,1,0,Blur,Your hero can't take damage this turn.,"{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,0,0
1,1,14,,4,2,2,1,Shadowhoof Slayer,<b>Battlecry:</b> Give your hero +1&nbsp;Attac...,"{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,0,0
2,1,14,,4,1414,1,1,Crimson Sigil Runner,<b>Outcast:</b> Draw a card.,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,14,14 3,5,1443,3,1,Demon Companion,Summon a random Demon Companion.,"{'relevant': True, 'constructed': True}",...,0,1,0,0,0,0,0,0,0,0
4,1,14,,5,1443,1,1,Double Jump,Draw an <b>Outcast</b> card from your deck.,"{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,1,12,,4,1466,5,10,"C'Thun, the Shattered",<b>Start of Game:</b> Break into pieces. <b>Ba...,"{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,0,1
1285,1,12,,4,1466,4,10,Darkmoon Rabbit,"<b>Rush</b>, <b>Poisonous</b> Also damages the...","{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,0,1
1286,1,12,,4,1466,5,10,"N'Zoth, God of the Deep",<b>Battlecry:</b> Resurrect a friendly minion ...,"{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,0,1
1287,1,12,,4,1466,5,10,"Y'Shaarj, the Defiler",<b>Battlecry:</b> Add a copy of each <b>Corrup...,"{'relevant': True, 'constructed': True}",...,0,0,0,0,0,0,0,0,0,1


In [49]:
classes

,slug,id,name,cardId
0,demonhunter,14,Demon Hunter,56550.0
1,druid,2,Druid,274.0
2,hunter,3,Hunter,31.0
3,mage,4,Mage,637.0
4,paladin,5,Paladin,671.0
5,priest,6,Priest,813.0
6,rogue,7,Rogue,930.0
7,shaman,8,Shaman,1066.0
8,warlock,9,Warlock,893.0
9,warrior,10,Warrior,7.0


# Addressing Null Values

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1289 entries, 0 to 1288
Data columns (total 67 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   collectible          1289 non-null   int64  
 1   classId              1289 non-null   int64  
 2   multiClassIds        1289 non-null   object 
 3   cardTypeId           1289 non-null   int64  
 4   cardSetId            1289 non-null   int64  
 5   rarityId             1289 non-null   int64  
 6   manaCost             1289 non-null   int64  
 7   name                 1289 non-null   object 
 8   text                 1271 non-null   object 
 9   duels                708 non-null    object 
 10  minionTypeId         321 non-null    float64
 11  health               825 non-null    float64
 12  attack               860 non-null    float64
 13  childIds             295 non-null    object 
 14  durability           48 non-null     float64
 15  armor                6 non-null      f

### Variable: text

In [18]:
# checking values in text box
df.text.value_counts()

<b>Taunt</b>                                                                           15
<b>Charge</b>                                                                           7
<b>Stealth</b>                                                                          6
<b>Spell Damage +1</b>                                                                  6
<b>Divine Shield</b>                                                                    5
                                                                                       ..
Deal 2 damage to an undamaged minion.                                                   1
<b>Battlecry:</b> Summon a 2/2&nbsp;Squire.                                             1
<b>Choose One -</b> +5 Attack; or +5 Health and <b>Taunt</b>.                           1
<b>Battlecry:</b> Give your other minions "<b>Deathrattle:</b> Summon a 1/1 Demon."     1
After a friendly Mech dies, add a random Mech to your hand.                             1
Name: text

In [19]:
# filling null text values with 'no effect'
df["text"].fillna("no_effect", inplace = True) 

### Variable: duels

In [20]:
# checking duels values
df.duels.value_counts(dropna = False)

{'relevant': True, 'constructed': True}    708
NaN                                        581
Name: duels, dtype: int64

In [21]:
# updating duels column so that cards that were allowed in duels have value of 1 and 0 otherwise
df['duels'] = np.where((df.duels == "{'relevant': True, 'constructed': True}"), 1, 0)

### Variable: minionTypeId

In [22]:
# checking duels values
df.minionTypeId.value_counts(dropna = False)

NaN     968
20.0     75
15.0     63
18.0     51
24.0     50
17.0     32
14.0     25
23.0     18
21.0      6
26.0      1
Name: minionTypeId, dtype: int64

In [23]:
# converting nulls to 'neutral' type
df['minionTypeId'] = np.where((df.minionTypeId.isnull() == True), 'Neutral', df.minionTypeId)

### Variables: health, attack, durability, armor
All of these variables respective columns have null values since none of these variables apply to every card (examples: only minions have health while only weapons have durability). Ill be leaving the nulls in these particular columns for the reasons listed below.

- I don't want to impute a string such as 'NA' as this will alter the numeric data type of the column
- I don't want to impute a false numeric value as the null state is more accurate in this instance
- I'll be isolating the appropriate card types when exploring this variable so the nulls won't impact exploration
- This project will not include modeling so leaving nulls in these columns won't be an issue for modeling either

However, I will be checking for instances of attributes being applied to the wrong card types and correcting them as needed. For example, a weapon having health or a minion having durability.

### Variable: childIds

In [24]:
# checking duels values
df.childIds.value_counts(dropna = False)

NaN                                                                       994
[52897, 52900, 53160, 53161, 53162, 53163, 55378, 60588, 64652, 64653]     15
[59723]                                                                     8
[53921]                                                                     3
[56927]                                                                     3
                                                                         ... 
[54541]                                                                     1
[325, 451]                                                                  1
[55778]                                                                     1
[61272]                                                                     1
[53310]                                                                     1
Name: childIds, Length: 261, dtype: int64

In [25]:
# filling nulls with "no_childid"
df.childIds.fillna("no_childid", inplace = True) 